In [ ]:
import ipywidgets as widgets
from bokeh.models import ColumnDataSource, DataTable, TableColumn
from bokeh.embed import components
from bokeh.resources import CDN
from IPython.display import display, HTML, clear_output
import pandas as pd

def crear_tabla_html(df):
    source = ColumnDataSource(df)
    columns = [TableColumn(field=col, title=col.capitalize()) for col in df.columns]
    tabla = DataTable(source=source, columns=columns, width=600, height=280)
    script, div = components(tabla)
    return CDN.render() + div + script

# Ejemplos tablas
df1 = pd.DataFrame({"Nombre": ["Ana", "Luis"], "Edad": [23, 30]})
df2 = pd.DataFrame({"Nombre": ["Carlos", "María"], "Edad": [45, 29]})

tablas_html = {
    "Tabla 1": crear_tabla_html(df1),
    "Tabla 2": crear_tabla_html(df2),
}

selector = widgets.Select(
    options=list(tablas_html.keys()),
    description="Selecciona tabla:",
    layout=widgets.Layout(width='220px'),
    rows=3
)

output = widgets.Output()

def on_change(change):
    if change['name'] == 'value' and change['type'] == 'change':
        output.clear_output()
        with output:
            display(HTML(tablas_html[change['new']]))

# Mostrar tabla inicial
with output:
    display(HTML(tablas_html[selector.value]))

selector.observe(on_change)

display(widgets.HBox([selector, output]))


In [ ]:
import numpy as np
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource
from bokeh.embed import components
from bokeh.resources import CDN
from ipywidgets import HBox, VBox,widgets
from IPython.display import display

#[1]==================== Grafica Dividida =================================
x = np.linspace(0, 10, 200)
y1 = np.sin(x) + np.random.normal(scale=0.2, size=200)
y2 = np.cos(x) + np.random.normal(scale=0.2, size=200)

#[2]==================== Grafica Dividida =================================
#como se encuentra cada uno
estado = {
    "modo_oscuro": False,
    "mostrar_seno": True,
    "mostrar_coseno": True,
    "linea_punteada": False,
    "mostrar_leyenda": True
}

#[3]==================== Grafica Dividida =================================
#aqui se encuentran las casillas
def crear_figura(estado):
    modo_oscuro = estado["modo_oscuro"]
    mostrar_seno = estado["mostrar_seno"]
    mostrar_coseno = estado["mostrar_coseno"]
    linea_punteada = estado["linea_punteada"]
    mostrar_leyenda = estado["mostrar_leyenda"]

    p = figure(title="Gráfica con dos líneas", height=350, width=900,
               background_fill_color="#2F2F2F" if modo_oscuro else "white",
               border_fill_color="#2F2F2F" if modo_oscuro else "white",
               outline_line_color="white" if modo_oscuro else "black",
               x_axis_label='x', y_axis_label='y')

    line_dash = "dashed" if linea_punteada else "solid"

    legend_items = 0

    if mostrar_seno:
        source_line1 = ColumnDataSource(data=dict(x=x, y=y1))
        p.line('x', 'y', source=source_line1, line_width=2,
               color="cyan" if modo_oscuro else "blue",
               legend_label="Seno + ruido",
               line_dash=line_dash)
        legend_items += 1
    if mostrar_coseno:
        source_line2 = ColumnDataSource(data=dict(x=x, y=y2))
        p.line('x', 'y', source=source_line2, line_width=2,
               color="orange" if modo_oscuro else "red",
               legend_label="Coseno + ruido",
               line_dash=line_dash)
        legend_items += 1

    if legend_items > 0 and mostrar_leyenda:
        p.legend.location = "top_right"
        p.legend.visible = True
    else:
        p.legend.visible = False

    p.xaxis.axis_label_text_color = "white" if modo_oscuro else "black"
    p.yaxis.axis_label_text_color = "white" if modo_oscuro else "black"
    p.title.text_color = "white" if modo_oscuro else "black"
    p.xaxis.major_label_text_color = "white" if modo_oscuro else "black"
    p.yaxis.major_label_text_color = "white" if modo_oscuro else "black"

    p.xgrid.visible = not modo_oscuro
    p.ygrid.visible = not modo_oscuro

    return p


#[4]==================== Grafica Dividida =================================
def actualizar(change=None):
    p = crear_figura(estado)
    script, div = components(p)
    recursos = CDN.render()

    html_full = f"""
    {recursos}
    {div}
    {script}
    """

    html_escapado = html_full.replace('"', '&quot;').replace('\n', ' ')

    iframe_html = f"""
    <iframe width="920" height="380" frameborder="0"
        srcdoc="{html_escapado}">
    </iframe>
    """

    iframe.value = iframe_html

iframe = widgets.HTML()

#[5]==================== Grafica Dividida =================================
def on_change(change):
    estado["modo_oscuro"] = checkbox_modo_oscuro.value
    estado["mostrar_seno"] = checkbox_mostrar_seno.value
    estado["mostrar_coseno"] = checkbox_mostrar_coseno.value
    estado["linea_punteada"] = checkbox_linea_punteada.value
    estado["mostrar_leyenda"] = checkbox_mostrar_leyenda.value
    actualizar()
#[5]==================== Grafica Dividida =================================
#los que estan seleccionados

# Los checkboxes
checkbox_modo_oscuro = widgets.Checkbox(value=False, description="Modo oscuro")
checkbox_mostrar_seno = widgets.Checkbox(value=True, description="Mostrar línea seno")
checkbox_mostrar_coseno = widgets.Checkbox(value=True, description="Mostrar línea coseno")
checkbox_linea_punteada = widgets.Checkbox(value=False, description="Líneas punteadas")
checkbox_mostrar_leyenda = widgets.Checkbox(value=True, description="Mostrar leyenda")

checkbox_modo_oscuro.observe(on_change, names='value')
checkbox_mostrar_seno.observe(on_change, names='value')
checkbox_mostrar_coseno.observe(on_change, names='value')
checkbox_linea_punteada.observe(on_change, names='value')
checkbox_mostrar_leyenda.observe(on_change, names='value')

#[5]==================== Grafica Dividida =================================
# CONTROLES
controles = widgets.VBox([
    checkbox_modo_oscuro,
    checkbox_mostrar_seno,
    checkbox_mostrar_coseno,
    checkbox_linea_punteada,
    checkbox_mostrar_leyenda
])

#[5]==================== Grafica Dividida =================================
# Pestaña anidada (sub-tabs) con controles
inner_tabs = widgets.Tab(children=[controles,controles,controles,controles])
inner_tabs.set_title(0, "Cargar archivo")
inner_tabs.set_title(1, "Tabla")
inner_tabs.set_title(2, "Filtros")
inner_tabs.set_title(3, "Edicion grafica")

#[6]==================== Grafica Dividida =================================
# Contenido pestaña 1 con gráfica y sub-pestañas
tab1_content = HBox([         # Usamos HBox para layout horizontal
    VBox([                    # En una columna a la izquierda
        widgets.Label("Ventana 1 - Gráfica con controles"),
        iframe
    ]),
    inner_tabs               # Controles a la derecha
])

tab2_content = widgets.Label("Ventana 2 - contenido adicional")
#[6]==================== Grafica Dividida =================================
#ventana general
tabs = widgets.Tab(children=[tab1_content, tab2_content])
tabs.set_title(0, "Ventana 1 (Gráfica + control)")
tabs.set_title(1, "Ventana 2")

actualizar()
display(tabs)


ValueError: tuple.index(x): x not in tuple

In [ ]:
panel = widgets.HBox([

])

In [ ]:
from bokeh.plotting import figure
from bokeh.models import Tabs, TabPanel, DataTable, TableColumn, ColumnDataSource
from bokeh.layouts import row
from bokeh.embed import components
from bokeh.resources import CDN
from IPython.display import display, HTML

# Gráfico fijo
p = figure(width=400, height=300, title="Gráfico fijo")
p.line([1, 2, 3], [4, 5, 6])
tab1 = TabPanel(child=p, title="Gráfico fijo")

# Función para crear tabla con n filas
def crear_tabla(n):
    data = dict(name=[f"Item {i}" for i in range(n)], value=list(range(n)))
    source = ColumnDataSource(data)
    columns = [TableColumn(field="name", title="Nombre"), TableColumn(field="value", title="Valor")]
    return DataTable(source=source, columns=columns, width=400, height=280)

# Crear varias tablas
tabla1 = crear_tabla(30)
tabla2 = crear_tabla(50)
tabla3 = crear_tabla(70)

# Crear tabs internos para las tablas
tab_tabla1 = TabPanel(child=tabla1, title="Tabla 1")
tab_tabla2 = TabPanel(child=tabla2, title="Tabla 2")
tab_tabla3 = TabPanel(child=tabla3, title="Tabla 3")

tabs_tablas = Tabs(tabs=[tab_tabla1, tab_tabla2, tab_tabla3])
tabs_tablas.width = 420
tabs_tablas.height = 320
tabs_tablas.sizing_mode = "fixed"
tabs_tablas.css_classes = ["scrollable-tab"]

# Tab con tablas múltiples
tab3 = TabPanel(child=tabs_tablas, title="Tablas múltiples")

# Crear Tabs principales
tabs1 = Tabs(tabs=[tab1])
tabs1.width = 420
tabs1.height = 320
tabs1.sizing_mode = "fixed"

tabs3_main = Tabs(tabs=[tab3])
tabs3_main.width = 420
tabs3_main.height = 320
tabs3_main.sizing_mode = "fixed"

# CSS para scroll dentro del tab de tablas
css = """
<style>
.scrollable-tab .bk-panel > .bk-root {
  overflow: auto;
  max-height: 320px;
}
</style>
"""
display(HTML(css))

# Layout lado a lado
layout = row(tabs1, tabs3_main)

script, div = components(layout)
recursos = CDN.render()
display(HTML(recursos + div + script))


In [ ]:
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import CheckboxGroup, CustomJS
from bokeh.layouts import column

output_notebook()

# Crear la figura y una línea
p = figure(title="Ocultar/Mostrar línea", width=500, height=300)
linea = p.line([1, 2, 3], [4, 5, 6], color="blue", line_width=2)

# Una sola casilla
checkbox = CheckboxGroup(labels=["Mostrar línea"], active=[0])

# Callback en JavaScript
callback = CustomJS(args=dict(linea=linea), code="""
    linea.visible = cb_obj.active.includes(0);
""")

checkbox.js_on_change("active", callback)

# Mostrar en Colab
layout = column(checkbox, p)
show(layout)


In [ ]:
from bokeh.plotting import figure, output_notebook, show
from bokeh.layouts import column, row
from bokeh.models import Slider, Select, CheckboxGroup, TextInput, TabPanel, Tabs
from bokeh.models import CustomJS
from bokeh.models.widgets import Button

output_notebook()

# ====== 1. Figura ======
p = figure(width=400, height=300, title="Ejemplo de contenedores")
linea = p.line([1, 2, 3, 4], [2, 5, 8, 2], line_width=2, color="navy")

# ====== 2. Widgets ======
slider = Slider(start=0, end=10, value=5, step=1, title="Desplazamiento")
select = Select(title="Color:", value="navy", options=["navy", "red", "green"])
checkbox = CheckboxGroup(labels=["Mostrar línea"], active=[0])
text_input = TextInput(value="Título inicial", title="Título gráfico:")
boton = Button(label="Click aquí", button_type="success")

# ====== 3. Callbacks JavaScript ======
slider.js_on_change("value", CustomJS(args=dict(linea=linea), code="""
    const dy = cb_obj.value;
    linea.data_source.data.y = [2+dy, 5+dy, 8+dy, 2+dy];
    linea.data_source.change.emit();
"""))

select.js_on_change("value", CustomJS(args=dict(linea=linea), code="""
    linea.glyph.line_color = cb_obj.value;
"""))

checkbox.js_on_change("active", CustomJS(args=dict(linea=linea), code="""
    linea.visible = cb_obj.active.includes(0);
"""))

text_input.js_on_change("value", CustomJS(args=dict(figura=p), code="""
    figura.title.text = cb_obj.value;
"""))

boton.js_on_click(CustomJS(code="alert('¡Botón presionado!')"))

# ====== 4. Contenedores ======
# Columna de controles
col_controles = column(slider, select, checkbox, text_input, boton, width=250)

# Fila: Controles + Gráfico
fila_principal = row(col_controles, p)

# Tabs para organizar
tab1 = TabPanel(child=fila_principal, title="Interacción")
tab2 = TabPanel(child=column(TextInput(value="Otro tab")), title="Otro Tab")

tabs = Tabs(tabs=[tab1, tab2])

# ====== 5. Mostrar ======
show(tabs)


In [ ]:
from bokeh.plotting import output_notebook, show
from bokeh.models import ColumnDataSource, DataTable, TableColumn, Select, CustomJS
from bokeh.layouts import column
import pandas as pd

output_notebook()

# Crear varias tablas (dataframes)
def crear_df(n):
    return pd.DataFrame({
        "Número": list(range(1, n+1)),
        "Texto": [f"Fila {i}" for i in range(1, n+1)]
    })

dfs = [crear_df(10), crear_df(15), crear_df(20)]

# Crear ColumnDataSources para cada dataframe
sources = [ColumnDataSource(df) for df in dfs]

# Columnas comunes para todas las tablas
columns = [
    TableColumn(field="Número", title="Número"),
    TableColumn(field="Texto", title="Texto"),
]

# Crear DataTable con la primera tabla
data_table = DataTable(source=sources[0], columns=columns, width=400, height=300)

tabla_layout =column(*tablas)
# Selector para elegir tabla
select = Select(title="Seleccione tabla:", value="Tabla 1", options=[f"Tabla {i+1}" for i in range(len(dfs))])

# Callback JS para cambiar el data source según la selección
select.js_on_change("value", CustomJS(args=dict(sources=sources, table=data_table), code="""
    const index = parseInt(cb_obj.value.split(' ')[1]) - 1;
    table.source.data = sources[index].data;
    table.source.change.emit();
"""))

layout = column(select, data_table)

tab1 = TabPanel(child=p, title="Gráfico")#Usar child=... porque TabPanel todavía acepta esa sintaxis
tab2 = TabPanel(child=layout, title="Tabla",disabled=False)
tab3 = TabPanel(child=controles, title="Controles")

#tab4 = TabPanel(child=checkbox, title="Cargar")

tabs = Tabs(tabs=[tab1])
#HERRAMIENTAS
tabs3 = Tabs(tabs=[tab3,tab2])
tabs3.width = 400    # ancho fijo para todo el conjunto de tabs
tabs3.height = 100   # alto fijo
general = row(tabs3,tabs)

script, div = components(general)

recursos = CDN.render()
display(HTML(recursos + div + script))


In [ ]:
from bokeh.plotting import output_notebook, show
from bokeh.models import DataTable, TableColumn, ColumnDataSource, Select, CustomJS
from bokeh.layouts import column
import pandas as pd

output_notebook()

def crear_tabla(n):
    df = pd.DataFrame({
        "Número": list(range(1, n+1)),
        "Texto": [f"Fila {i}" for i in range(1, n+1)]
    })
    source = ColumnDataSource(df)
    columns = [
        TableColumn(field="Número", title="Número"),
        TableColumn(field="Texto", title="Texto")
    ]
    return DataTable(source=source, columns=columns, width=400, height=200)

N = 5
tablas = [crear_tabla(5 + i) for i in range(N)]

# Asignamos un id único a cada tabla
for i, tabla in enumerate(tablas):
    tabla.name = f"tabla_{i}"
    tabla.visible = (i == 0)  # Solo visible la primera tabla

tabla_layout = column(*tablas)

options = [f"Tabla {i+1}" for i in range(N)]
select = Select(title="Seleccione tabla", value=options[0], options=options, width=200)

# JS callback para cambiar visibilidad
callback = CustomJS(args=dict(tabs=tablas, select=select), code="""
    const selected = select.value;
    for(let i=0; i<tabs.length; i++){
        if(selected === `Tabla ${i+1}`){
            tabs[i].visible = true;
        } else {
            tabs[i].visible = false;
        }
    }
""")

select.js_on_change('value', callback)

layout_final = column(select, tabla_layout)
show(layout_final)
